In [11]:
import numpy as np
import pandas as pd
import json
import sklearn as sk
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')


In [12]:
def confusion_matrix(data, prompt):
    data = data[data["PromptType"]==prompt]
    return np.array([[len(data[data["model_bruteforce"]=="1"][data["human_bruteforce"]=="1"]), len(data[data["model_bruteforce"]=="1"][data["human_bruteforce"]=="0"])], [len(data[data["model_bruteforce"]=="0"][data["human_bruteforce"]=="1"]), len(data[data["model_bruteforce"]=="0"][data["human_bruteforce"]=="0"])]])

def correctness(data, prompt):
    data = data[data["PromptType"]==prompt]
    return len(data[data["correctness"]=="1"]) / len(data)

In [13]:
alldata = {}
models = ['DSChat', 'DSReason', 'GeminiFlash', 'o3', 'Qwen1', 'Qwen14', 'Qwen70']
tests = ['MathMain', 'MathHint', 'MathHintCombined', 'LogicMain', 'LogicHint', 'LogicHintCombined']
humanbruteforcemath = {}
humanbruteforcetotalmath = {}
humanbruteforcelogic = {}
humanbruteforcetotallogic = {}
mathdifficulty = {}
logicdifficulty = {}
mathpopularity = {}
logicpopularity = {}
mathcategory = {}
logiccategory = {}

logicdata = pd.read_csv("data/braingle/braingle_Logic_with_categories.csv")
mathdata = pd.read_csv("data/braingle/braingle_Math_with_categories.csv")
for index, row in logicdata.iterrows():
    logicdifficulty[index] = row["Difficulty"]
    logicpopularity[index] = row["Popularity/Fun"]
    logiccategory[index] = row['categories']
for index, row in mathdata.iterrows():
    mathdifficulty[index] = row["Difficulty"]
    mathpopularity[index] = row["Popularity/Fun"]
    mathcategory[index] = row['categories']

# models = ['FinalLogic-Qwen1']
for model in models:
    file_path = f"response_evaluation/Logic/HintRedo-{model}/resultsEvaluations_evaluatedbyo3-2025-04-16.jsonl"
    with open(file_path, 'r') as file:
        alldata[(model, "LogicHint")] = [json.loads(line) for line in file]

    for index, row in pd.DataFrame(alldata[(model, "LogicHint")]).iterrows():
        try:
            if (row['human_bruteforce'] != '1' and row['human_bruteforce'] != '0'):
                continue
            if (row["ID"] not in humanbruteforcetotallogic.keys()):
                humanbruteforcetotallogic[row["ID"]] = 0
                humanbruteforcelogic[row["ID"]] = 0
            humanbruteforcetotallogic[row["ID"]] += 1
            humanbruteforcelogic[row["ID"]] += int(row['human_bruteforce'])
        except:
            pass

for model in models:
    file_path = f"response_evaluation/Logic/FinalLogic-{model}/resultsEvaluations_evaluatedbyo3-2025-04-16.jsonl"
    with open(file_path, 'r') as file:
        alldata[(model, "LogicMain")] = [json.loads(line) for line in file]

    for index, row in pd.DataFrame(alldata[(model, "LogicMain")]).iterrows():
        try:
            if (row['human_bruteforce'] != '1' and row['human_bruteforce'] != '0'):
                continue
            if (row["ID"] not in humanbruteforcetotallogic.keys()):
                humanbruteforcetotallogic[row["ID"]] = 0
                humanbruteforcelogic[row["ID"]] = 0
            humanbruteforcetotallogic[row["ID"]] += 1
            humanbruteforcelogic[row["ID"]] += int(row['human_bruteforce'])
        except:
            pass

for model in models:
    file_path = f"response_evaluation/Math/FinalMath-{model}/resultsEvaluations_evaluatedbyo3-2025-04-16.jsonl"
    with open(file_path, 'r') as file:
        alldata[(model, "MathMain")] = [json.loads(line) for line in file]

    for index, row in pd.DataFrame(alldata[(model, "MathMain")]).iterrows():
        try:
            if (row['human_bruteforce'] != '1' and row['human_bruteforce'] != '0'):
                continue
            if (row["ID"] not in humanbruteforcetotalmath.keys()):
                humanbruteforcetotalmath[row["ID"]] = 0
                humanbruteforcemath[row["ID"]] = 0
            humanbruteforcetotalmath[row["ID"]] += 1
            humanbruteforcemath[row["ID"]] += int(row['human_bruteforce'])
        except:
            pass

for model in models:
    file_path = f"response_evaluation/Math/HintRedo-{model}/resultsEvaluations_evaluatedbyo3-2025-04-16.jsonl"
    with open(file_path, 'r') as file:
        alldata[(model, "MathHint")] = [json.loads(line) for line in file]

    for index, row in pd.DataFrame(alldata[(model, "MathHint")]).iterrows():
        try:
            if (row['human_bruteforce'] != '1' and row['human_bruteforce'] != '0'):
                continue
            if (row["ID"] not in humanbruteforcetotalmath.keys()):
                humanbruteforcetotalmath[row["ID"]] = 0
                humanbruteforcemath[row["ID"]] = 0
            humanbruteforcetotalmath[row["ID"]] += 1
            humanbruteforcemath[row["ID"]] += int(row['human_bruteforce'])
        except:
            pass

for model in models:
    file_path = f"response_evaluation/Logic/CombinedHint-{model}/resultsEvaluations_evaluatedbyo3-2025-04-16.jsonl"
    try:
        with open(file_path, 'r') as file:
            alldata[(model, "LogicHintCombined")] = [json.loads(line) for line in file]

        for index, row in pd.DataFrame(alldata[(model, "LogicHintCombined")]).iterrows():
            try:
                if (row['human_bruteforce'] != '1' and row['human_bruteforce'] != '0'):
                    continue
                if (row["ID"] not in humanbruteforcetotallogic.keys()):
                    humanbruteforcetotallogic[row["ID"]] = 0
                    humanbruteforcelogic[row["ID"]] = 0
                humanbruteforcetotallogic[row["ID"]] += 1
                humanbruteforcelogic[row["ID"]] += int(row['human_bruteforce'])
            except:
                pass
    except:
        pass

for model in models:
    file_path = f"response_evaluation/Math/CombinedHint-{model}/resultsEvaluations_evaluatedbyo3-2025-04-16.jsonl"
    try:
        with open(file_path, 'r') as file:
            alldata[(model, "MathHintCombined")] = [json.loads(line) for line in file]

        for index, row in pd.DataFrame(alldata[(model, "MathHintCombined")]).iterrows():
            try:
                if (row['human_bruteforce'] != '1' and row['human_bruteforce'] != '0'):
                    continue
                if (row["ID"] not in humanbruteforcetotalmath.keys()):
                    humanbruteforcetotalmath[row["ID"]] = 0
                    humanbruteforcemath[row["ID"]] = 0
                humanbruteforcetotalmath[row["ID"]] += 1
                humanbruteforcemath[row["ID"]] += int(row['human_bruteforce'])
            except:
                pass
    except:
        pass


for i in range(250):
    humanbruteforcemath[i] = round(humanbruteforcemath[i] / humanbruteforcetotalmath[i])
    humanbruteforcelogic[i] = round(humanbruteforcelogic[i] / humanbruteforcetotallogic[i])

print(humanbruteforcemath)
print(humanbruteforcelogic)

{0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 1, 10: 0, 11: 0, 12: 1, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 1, 28: 0, 29: 0, 30: 0, 31: 0, 32: 1, 33: 0, 34: 0, 35: 0, 36: 0, 37: 0, 38: 1, 39: 0, 40: 0, 41: 0, 42: 0, 43: 1, 44: 1, 45: 1, 46: 1, 47: 0, 48: 0, 49: 0, 50: 0, 51: 0, 52: 0, 53: 0, 54: 1, 55: 1, 56: 0, 57: 0, 58: 0, 59: 0, 60: 0, 61: 0, 62: 0, 63: 1, 64: 0, 65: 0, 66: 0, 67: 0, 68: 0, 69: 0, 70: 0, 71: 0, 72: 1, 73: 0, 74: 0, 75: 0, 76: 0, 77: 0, 78: 0, 79: 0, 80: 0, 81: 1, 82: 0, 83: 0, 84: 0, 85: 0, 86: 0, 87: 0, 88: 0, 89: 0, 90: 0, 91: 0, 92: 1, 93: 0, 94: 0, 95: 0, 96: 0, 97: 1, 98: 0, 99: 0, 100: 0, 101: 0, 102: 0, 103: 1, 104: 1, 105: 0, 106: 0, 107: 1, 108: 0, 109: 0, 110: 0, 111: 0, 112: 0, 113: 0, 114: 0, 115: 0, 116: 0, 117: 0, 118: 0, 119: 0, 120: 0, 121: 0, 122: 0, 123: 0, 124: 0, 125: 0, 126: 0, 127: 0, 128: 0, 129: 1, 130: 0, 131: 0, 132: 1, 133: 1, 134: 0, 135: 1, 136: 0, 137: 0, 138: 

In [14]:
for model in models:
    for test in tests:
        try:
            data = pd.DataFrame(alldata[(model, test)])
        except:
            continue
        prompts = data["PromptType"].unique()

        for prompt in prompts:
            if "hint" in prompt and "Main" in test or "symbol" in prompt:
                continue
            tempdata = data[data["PromptType"]==prompt]
            bfarray = np.zeros((2, 2))
            bfdiff = []
            nbfdiff = []
            bfpop = []
            nbfpop = []
            correctness = []
            correctnessDiff = []
            category = {}
            count = 0

            for index, row in tempdata.iterrows():
                # if (type(row["Response"]) != str):
                #     print(row["Response"])
                if (type(row["Response"]) != str or row["Response"] == None or row["Response"] == "NaN" or row["Response"] == "None" or row["Response"] == "" or row["model_bruteforce"] == "NULL" or row["Response"] is str and row["Response"].isspace()):
                    # print("Filtered!")
                    continue
                try:
                    if (row["model_bruteforce"] == "1"):
                        if "Math" in test:
                            bfdiff.append(mathdifficulty[row["ID"]])
                            bfpop.append(mathpopularity[row["ID"]])
                        else:
                            bfdiff.append(logicdifficulty[row["ID"]])
                            bfpop.append(logicpopularity[row["ID"]])
                    elif (row["model_bruteforce"] == "0"):
                        if "Math" in test:
                            nbfdiff.append(mathdifficulty[row["ID"]])
                            nbfpop.append(mathpopularity[row["ID"]])
                        else:
                            nbfdiff.append(logicdifficulty[row["ID"]])
                            nbfpop.append(logicpopularity[row["ID"]])
                    
                    if "Math" in test:
                        bfarray[1-int(row["model_bruteforce"])][1-humanbruteforcemath[row['ID']]] += 1
                    if "Logic" in test:
                        bfarray[1-int(row["model_bruteforce"])][1-humanbruteforcelogic[row['ID']]] += 1
                    correctness.append(int(row["correctness"]))

                    if "Logic" in test:
                        # print(logiccategory[row["ID"]])
                        if logiccategory[row["ID"]] not in category.keys():
                            category[logiccategory[row["ID"]]] = [0, 0]
                        
                        # print(row["model_bruteforce"])

                        category[logiccategory[row["ID"]]][1-int(row["model_bruteforce"])] += 1

                    if (row['ID'] < 50):
                        correctnessDiff.append(int(row["correctness"]))
                    # count += 1

                    
                except Exception as e:
                    # print("Error:", e)
                    # print(row["model_bruteforce"])
                    pass
            # print(category)
            
            print(f"Model: {model}, Test: {test}, Prompt: {prompt}", 
                #   print(category),
                  np.round(100*bfarray.flatten()/np.sum(bfarray), 1), np.sum(bfarray), 
                  "Correctness:", str(np.round(np.mean(correctness), 3)) + "/" + str(np.round(np.mean(correctnessDiff), 3)), 
                #   "Difficulty (BF/NBF):", np.round(np.mean(bfdiff), 2), np.round(np.mean(nbfdiff), 2), 
                #   "Popularity (BF/NBF):", np.round(np.mean(bfpop), 2), np.round(np.mean(nbfpop), 2),
                  
            )

Model: DSChat, Test: MathMain, Prompt: basicprompt [10.4 30.8  2.8 56. ] 250.0 Correctness: 0.58/0.4
Model: DSChat, Test: MathMain, Prompt: mathPrompt [ 9.6 28.4  3.6 58.4] 250.0 Correctness: 0.536/0.32
Model: DSChat, Test: MathHint, Prompt: hintPrompt [ 9.6 28.8  3.6 58. ] 250.0 Correctness: 0.568/0.38
Model: DSChat, Test: MathHintCombined, Prompt: combinedhintPrompt [11.2 30.   2.  56.8] 250.0 Correctness: 0.6/0.38
Model: DSChat, Test: LogicMain, Prompt: basicprompt [ 6. 38.  4. 52.] 250.0 Correctness: 0.392/0.32
Model: DSChat, Test: LogicMain, Prompt: mathPrompt [ 7.2 34.   2.8 56. ] 250.0 Correctness: 0.412/0.34
Model: DSChat, Test: LogicHint, Prompt: hintPrompt [ 7.6 30.   2.4 60. ] 250.0 Correctness: 0.414/0.26
Model: DSChat, Test: LogicHintCombined, Prompt: combinedhintPrompt [ 6.4 28.8  3.6 61.2] 250.0 Correctness: 0.408/0.2
Model: DSReason, Test: MathMain, Prompt: basicprompt [ 8.  13.2  5.2 73.6] 250.0 Correctness: 0.664/0.48
Model: DSReason, Test: MathMain, Prompt: mathPromp

In [15]:
print("Human Solution Math:")
totaldiffbf = []
totaldiffnbf = []
totalpopbf = []
totalpopnbf = []
for i in range(250):
    if (humanbruteforcemath[i] == 1):
        totaldiffbf.append(mathdifficulty[i])
        totalpopbf.append(mathpopularity[i])
    else:
        totaldiffnbf.append(mathdifficulty[i])
        totalpopnbf.append(mathpopularity[i])
print("Difficulty (BF/NBF):", np.round(np.mean(totaldiffbf), 2), np.round(np.mean(totaldiffnbf), 2))
print("Popularity (BF/NBF):", np.round(np.mean(totalpopbf), 2), np.round(np.mean(totalpopnbf), 2))
print("Human Solution Logic:")
totaldiffbf = []
totaldiffnbf = []
totalpopbf = []
totalpopnbf = []
for i in range(250):
    if (humanbruteforcelogic[i] == 1):
        totaldiffbf.append(logicdifficulty[i])
        totalpopbf.append(logicpopularity[i])
    else:
        totaldiffnbf.append(logicdifficulty[i])
        totalpopnbf.append(logicpopularity[i])
print("Difficulty (BF/NBF):", np.round(np.mean(totaldiffbf), 2), np.round(np.mean(totaldiffnbf), 2))
print("Popularity (BF/NBF):", np.round(np.mean(totalpopbf), 2), np.round(np.mean(totalpopnbf), 2))

Human Solution Math:
Difficulty (BF/NBF): 2.83 2.8
Popularity (BF/NBF): 2.32 2.33
Human Solution Logic:
Difficulty (BF/NBF): 2.7 2.65
Popularity (BF/NBF): 2.37 2.51


In [16]:
mathcategory

{0: '4,7',
 1: '2',
 2: '5',
 3: '1',
 4: '2',
 5: '2,8',
 6: '4',
 7: '4',
 8: '2,8',
 9: '8',
 10: '2',
 11: '2',
 12: '1',
 13: '9',
 14: '9',
 15: '2,8',
 16: '2,8',
 17: '7',
 18: '2',
 19: '2',
 20: '3',
 21: '2',
 22: '1',
 23: '2',
 24: '5',
 25: '1',
 26: '1,2',
 27: '2',
 28: '5',
 29: '1',
 30: '9',
 31: '8',
 32: '2',
 33: '2',
 34: '4',
 35: '4',
 36: '2',
 37: '2',
 38: '2',
 39: '3',
 40: '5',
 41: '2',
 42: '2',
 43: '2,7',
 44: '8',
 45: '7',
 46: '3',
 47: '7',
 48: '4',
 49: '2',
 50: '4',
 51: '4,2',
 52: '2',
 53: '2,7',
 54: '2',
 55: '2',
 56: '1',
 57: '2,5',
 58: '4,9',
 59: '5',
 60: '8',
 61: '9',
 62: '2,5',
 63: '2,5',
 64: '6',
 65: '7',
 66: '1,8',
 67: '9,4',
 68: '2',
 69: '2',
 70: '2',
 71: '8',
 72: '8',
 73: '2',
 74: '7',
 75: '2',
 76: '2',
 77: '2',
 78: '6',
 79: '5',
 80: '1',
 81: '2',
 82: '4,7',
 83: '4',
 84: '3',
 85: '6',
 86: '8',
 87: '2',
 88: '8',
 89: '2',
 90: '2',
 91: '2',
 92: '3',
 93: '5',
 94: '2,8',
 95: '2',
 96: '5',
 97: '

In [17]:
for i in range(10):
    for j in range(250):
        if (str(i) not in mathcategory(j)):
            continue

TypeError: 'dict' object is not callable